<h1 style="color:Blue;">CIFAR-10 with TensorFlow</h1>

<b>
This example shows how to build a simple CNN for classifying the CIFAR10 dataset using the TensorFlow layers API, then how to train and evaluate it.</b>

<b>First, we import the necessary Python packages and print out the versions of the TensorFlow & Keras packages.</b>

In [ ]:
import os
import sys
import shutil
import tensorflow as tf
from tensorflow.python.tools import freeze_graph

print("Tensorflow version. ", tf.VERSION)
print("Keras version. ", tf.keras.__version__)

<b>Create some directories for the TensorBoard event logs and the checkpoints...</b>

In [ ]:
SCRIPT_DIR = os.getcwd()

GRAPH_FILE_BIN = 'graph.pb'
GRAPH_FILE_TXT = 'graph.pbtxt'
CHKPT_FILE = 'float_model.ckpt'
FROZEN_GRAPH_FILE = 'frozen_graph.pb'

CHKPT_DIR = os.path.join(SCRIPT_DIR, 'chkpts')
TB_LOG_DIR = os.path.join(SCRIPT_DIR, 'tb_logs')
FREEZE_DIR = os.path.join(SCRIPT_DIR, 'freeze')
CHKPT_PATH = os.path.join(CHKPT_DIR, CHKPT_FILE)

# create a directory for the TensorBoard data if it doesn't already exist
# delete it and recreate if it already exists
if (os.path.exists(TB_LOG_DIR)):
    shutil.rmtree(TB_LOG_DIR)
os.makedirs(TB_LOG_DIR)
print("Directory " , TB_LOG_DIR ,  "created ") 


# create a directory for the checkpoints if it doesn't already exist
# delete it and recreate if it already exists
if (os.path.exists(CHKPT_DIR)):
    shutil.rmtree(CHKPT_DIR)
os.makedirs(CHKPT_DIR)
print("Directory " , CHKPT_DIR ,  "created ") 


# create a directory for the frozen graph if it doesn't already exist
# delete it and recreate if it already exists
if (os.path.exists(FREEZE_DIR)):
    shutil.rmtree(FREEZE_DIR)
os.makedirs(FREEZE_DIR)
print("Directory " , FREEZE_DIR ,  "created ")

<b>Set up the learning rate for the Optimizer, the number of epochs and the batch size. Note that the number of epochs is et to a very low value so that the Notebook can be run quickly, this should really be set to a much higher number, perphas as ahigh as 10000.</b>

In [ ]:
LEARNRATE = 0.0001
EPOCHS = 3
BATCHSIZE = 50

<h2 style="color:Blue;">Data Wrangling</h2>

<b>
Download the CIFAR-10 dataset using the Keras function. What you get is a dataset that has been split into 50k images & labels for training, 10k images and labels for test. The 'images' are actually numpy arrays with the datatype set to 8bit unsigned integer. We scale this image data back to the range 0:1.0 by dividing by 255.0. The labels are also integers, so we one-hot encode them using the Keras 'to_categorical()' method.
</b>

In [ ]:
# CIFAR10 datset has 60k images. Training set is 50k, test set is 10k.
# Each image is 32x32 pixels RGB
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Scale image data from range 0:255 to range 0:1
x_train = x_train / 255.0
x_test = x_test / 255.0

# one-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# calculate total number of batches in the training data
total_batches = int(len(x_train)/BATCHSIZE)

<h2 style="color:Blue;">The Computational Graph</h2>

<b>
The placeholders for inputting data have shapes that match the modified datasets. The 'x' placeholder takes in the 32pixel x 32pixel RGB images (..actually numpy arrays..) and so has shape [None, 32, 32, 3].  The 'y' placeholder takes in the one-hot encoded labels.
</b>

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name='images_in')
y = tf.placeholder(tf.float32, [None, 10], name='labels_in')

<b>
Now we define our simple CNN as a series of layers..3 sets of 2D convolution and max pooling layers, then a flatten layer before a final fully connected layer with softmax activation and 10 outputs.
</b>

In [ ]:
def cnn(x):
  conv1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=3, kernel_initializer=tf.glorot_uniform_initializer(), activation=tf.nn.relu, name='conv1')
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, name='pool1')
  conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=3, kernel_initializer=tf.glorot_uniform_initializer(), activation=tf.nn.relu, name='conv2')
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2, name='pool2')
  conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=3, kernel_initializer=tf.glorot_uniform_initializer(), activation=tf.nn.relu, name='conv3')
  pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2, name='pool3')
  flat1 = tf.layers.flatten(inputs=pool3,name='flat1')
  fc1 = tf.layers.dense(inputs=flat1, units=1024, kernel_initializer=tf.glorot_uniform_initializer(), activation=tf.nn.relu, name='fc1')
  prediction = tf.layers.dense(inputs=fc1, units=10, kernel_initializer=tf.glorot_uniform_initializer(), activation=tf.nn.softmax, name='prediction')

  return prediction

# build the network, input comes from the 'x' placeholder
prediction = cnn(x)

<b>The loss function is a cross entropy function for classification which accepts labels in one-hot format (..which explains why we one-hot encoded the labels earlier..). The training optimizer is an Adaptive Momentum type.</b>

In [ ]:
# Define a cross entropy loss function
loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=prediction, onehot_labels=y))

# Define the optimizer function
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNRATE).minimize(loss)

# Check to see if predictions match the labels
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

 # Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

<b>We will collect the loss and accuracy data for displaying in TensorBoard along with the images that are fed into the 'x' placeholder.</b>

In [ ]:
# TensorBoard data collection
tf.summary.scalar('cross_entropy_loss', loss)
tf.summary.scalar('accuracy', accuracy)
tf.summary.image('input_images', x)

<b>We define an instance of a saver object which will be used inside our session to save the trained model checkpoint.</b>

In [ ]:
# set up saver object
saver = tf.train.Saver()

<h2 style="color:Blue;">The Session</h2>

<b>Inside the session, we initialize all the variables then loop through the number of epochs, sending the training data into the 'x' and 'y' placeholders.

When we exit the training loop, the final accuracy is calculated and then the final trained model is saved as a checkpoint and as a graph in a protobuf text file.
</b>

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())
    
    # TensorBoard writer
    writer = tf.summary.FileWriter(TB_LOG_DIR, sess.graph)
    tb_summary = tf.summary.merge_all()

    # Training cycle with training data
    for epoch in range(EPOCHS):
        print ("Epoch:", epoch)

        # process all batches
        for i in range(total_batches):
            
            # fetch a batch from training dataset
            batch_x, batch_y = x_train[i*BATCHSIZE:i*BATCHSIZE+BATCHSIZE], y_train[i*BATCHSIZE:i*BATCHSIZE+BATCHSIZE]

            # Run graph for optimization, loss, accuracy - i.e. do the training
            _, acc, s = sess.run([optimizer, accuracy, tb_summary], feed_dict={x: batch_x, y: batch_y})
            writer.add_summary(s, (epoch*total_batches + i))
            # Display accuracy per 100 batches
            if i % 100 == 0:
              print (" Batch:", i, 'Training accuracy: ', acc)

    print("Training Finished!")
    writer.flush()
    writer.close()

    # Evaluation cycle with test data
    print ("Final Accuracy with test set:", sess.run(accuracy, feed_dict={x: x_test[:1000], y: y_test[:1000]}))

    # save checkpoint & graph file as binary & text protobuf
    save_path = saver.save(sess, os.path.join(CHKPT_DIR, CHKPT_FILE) )
    tf.train.write_graph(sess.graph_def, CHKPT_DIR, GRAPH_FILE_BIN, as_text=False)
    tf.train.write_graph(sess.graph_def, CHKPT_DIR, GRAPH_FILE_TXT, as_text=True)

    # freeze the saved graph - converts variables to constants & removes training nodes
    freeze_graph.freeze_graph(input_graph=os.path.join(CHKPT_DIR,GRAPH_FILE_BIN),
                              input_saver='',
                              input_binary = True,
                              input_checkpoint = os.path.join(CHKPT_DIR, CHKPT_FILE),
                              output_node_names = 'prediction/Softmax',
                              restore_op_name ='save/restore_all',
                              filename_tensor_name = 'save/Const:0',
                              output_graph = os.path.join(FREEZE_DIR,FROZEN_GRAPH_FILE),
                              clear_devices = True,
                              initializer_nodes = '')


#  Session ended

print('FINISHED!')
print('Run `tensorboard --logdir=%s --port 6006 --host localhost` to see the results.' % TB_LOG_DIR)